# Function Calling

https://codelabs.developers.google.com/codelabs/gemini-function-calling?hl=en#1

# Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
PROJECT_ID = '[GCP-PROJECT-ID]'

# Install Dependencies

In [ ]:
!pip install --upgrade google-cloud-aiplatform # Vertex AI module

# Libraries

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
vertexai.init(project=PROJECT_ID)
model = GenerativeModel("gemini-pro")

# Basic Prompt

In [ ]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information, including current exchange rates. To get the most up-to-date exchange rate, you can check a currency converter or a financial news source.


# Specific Prompt

In [ ]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

```json
{
  "fromCurrency": "euros",
  "toCurrency": "US dollars"
}
```


# Function Calling

## Choosing API

In [ ]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2024-02-27","rates":{"AUD":1.6565,"BGN":1.9558,"BRL":5.3945,"CAD":1.465,"CHF":0.9544,"CNY":7.814,"CZK":25.332,"DKK":7.4551,"GBP":0.8562,"HKD":8.4947,"HUF":390.2,"IDR":16976,"ILS":3.9342,"INR":89.98,"ISK":149.3,"JPY":163.04,"KRW":1445.31,"MXN":18.5221,"MYR":5.1696,"NOK":11.431,"NZD":1.7601,"PHP":60.902,"PLN":4.3153,"RON":4.9668,"SEK":11.1805,"SGD":1.4584,"THB":38.932,"TRY":33.813,"USD":1.0856,"ZAR":20.772}}'

In [ ]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-pro")

## Function Declaration

In [ ]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

## Tool

In [ ]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

## Function Calling

In [ ]:
prompt = """What is the exchange rate from British Pounds to US Dollars?
How much is 500 British Pounds worth in US Dollars?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [ ]:
# print(response.candidates)
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "GBP"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "USD"
        }
      }
    }
  }
}



## Make an API Request

In [ ]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'GBP', 'to': 'USD', 'date': 'latest'}

In [ ]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"GBP","date":"2024-02-27","rates":{"USD":1.2679}}'

## Result

In [ ]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'The exchange rate from British Pounds to US Dollars on 2024-02-27 is 1.2679.\nThis means that 1 British Pound is worth 1.2679 US Dollars.\n\nTo convert 500 British Pounds to US Dollars, we multiply 500 by the exchange rate.\n\n500 * 1.2679 = 633.95 US Dollars\n\nTherefore, 500 British Pounds is worth 633.95 US Dollars.'